In [ ]:
import glob
import json
from collections import Counter
import pandas as pd

SKLEARN_DATA = "modules/sklearn_modules.json"
STATISTICS_DATA_DIR = "statistics/"

In [ ]:
with open(SKLEARN_DATA, "r", encoding="utf-8") as sklearn_file:
    sklearn_data = json.load(sklearn_file)

In [ ]:
with open("100DaysOfMLCodeChallenge_params.json", "r", encoding="utf-8") as project_file:
    project_data = json.load(project_file)

    for file_data in project_data:
        print("========")
        print(file_data)